# Examples for AbsLine class (v1.6)

In [1]:
# suppress warnings for these examples
import warnings
warnings.filterwarnings('ignore')

# import
import astropy.units as u
from linetools.spectralline import AbsLine, SpectralLine
from linetools import spectralline as ltsp
from linetools import line_utils as ltlu
from linetools.spectra import io as lsio
from linetools.spectra.xspectrum1d import XSpectrum1D

## Generate a line

In [2]:
abslin = AbsLine(1548.195*u.AA)
abslin

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


<AbsLine: CIV 1548, z=0.0000 wrest=1548.1950 Angstrom>

### Data

In [3]:
abslin.data

{'A': <Quantity 0.0 1 / s>,
 'Am': 0,
 'Ej': <Quantity 0.0 1 / cm>,
 'Ek': <Quantity 0.0 1 / cm>,
 'Ex': <Quantity 0.0 1 / cm>,
 'Jj': 0.0,
 'Jk': 0.0,
 'Ref': 'Verner1994',
 'Z': 6,
 'abundance': 8.4299999999999997,
 'col0': masked,
 'col7': masked,
 'el': 0,
 'f': 0.18999999761581421,
 'gamma': <Quantity 0.0 1 / s>,
 'gj': 2,
 'gk': 4,
 'group': 1,
 'ion': 4,
 'ion_correction': 0.0,
 'ion_name': u'CIV',
 'is_EUV': False,
 'is_HI': False,
 'is_Strong': True,
 'log(w*f)': 2.4685792560428057,
 'mol': '',
 'name': 'CIV 1548',
 'nj': 0,
 'nk': 0,
 'rel_strength': 10.898579256042805,
 'wrest': <Quantity 1548.195 Angstrom>}

### As dict

In [4]:
abslin = AbsLine(1548.195*u.AA)
tmp = abslin.to_dict()
tmp

{'analy': {u'datafile': u'',
  u'do_analysis': 1,
  u'flag_kin': 0,
  u'flg_eye': 0,
  u'flg_limit': 0,
  u'name': 'CIV 1548'},
 'attrib': {u'DEC': 0.0,
  u'EW': {'unit': u'Angstrom', 'value': 0.0},
  u'N': {'unit': u'1 / cm2', 'value': 0.0},
  u'RA': 0.0,
  u'b': {'unit': u'km / s', 'value': 0.0},
  u'flag_EW': 0,
  u'flag_N': 0,
  u'logN': 0.0,
  u'sig_EW': {'unit': u'Angstrom', 'value': 0.0},
  u'sig_N': {'unit': u'1 / cm2', 'value': 0.0},
  u'sig_b': {'unit': u'km / s', 'value': 0.0},
  u'sig_logN': 0.0},
 'data': {'A': {'unit': u'1 / s', 'value': 0.0},
  'Am': 0,
  'Ej': {'unit': u'1 / cm', 'value': 0.0},
  'Ek': {'unit': u'1 / cm', 'value': 0.0},
  'Ex': {'unit': u'1 / cm', 'value': 0.0},
  'Jj': 0.0,
  'Jk': 0.0,
  'Ref': 'Verner1994',
  'Z': 6,
  'abundance': 8.43,
  'el': 0,
  'f': 0.1899999976158142,
  'gamma': {'unit': u'1 / s', 'value': 0.0},
  'gj': 2,
  'gk': 4,
  'group': 1,
  'ion': 4,
  'ion_correction': 0.0,
  'ion_name': u'CIV',
  'is_EUV': False,
  'is_HI': False,
 

### From dict

In [5]:
tmp2 = SpectralLine.from_dict(tmp)
tmp2

<AbsLine: CIV 1548, z=0.0000 wrest=1548.1950 Angstrom>

### From old dict (using analy['vlim']) -- Backwards compatability

In [6]:
tmp.pop('limits')

{u'vlim': {'unit': u'km / s', 'value': [0.0, 0.0]},
 u'wrest': {'unit': u'Angstrom', 'value': 1548.195},
 u'wvlim': {'unit': u'Angstrom', 'value': [1548.195, 1548.195]},
 u'z': 0.0,
 u'zlim': [0.0, 0.0]}

In [7]:
tmp['analy']['vlim'] = [-150,150]*u.km/u.s
tmp['attrib']['z'] = 0.5

In [8]:
tmp3 = SpectralLine.from_dict(tmp)

## List of AbsLines

In [9]:
abslines = [abslin, AbsLine('HI 1215')]

### Parse

In [10]:
wrest_values = ltlu.parse_speclines(abslines, 'wrest')
wrest_values

[<Quantity 1548.195 Angstrom>, <Quantity 1215.67 Angstrom>]

### Transition Table

In [11]:
tbl = ltlu.transtable_from_speclines(abslines)
tbl

wrest,name,Z,ion,Ej,z,EW,sig_EW,flag_N,logN,sig_logN
Angstrom,,,,1 / cm,,Angstrom,Angstrom,,,
float64,str8,int64,int64,float64,float64,float64,float64,int64,float64,float64
1215.67,HI 1215,1,1,0.0,0.0,0.0,0.0,0,0.0,0.0
1548.195,CIV 1548,6,4,0.0,0.0,0.0,0.0,0,0.0,0.0


## Measure an EW

In [12]:
# Set spectrum
abslin.analy['spec'] = XSpectrum1D.from_file('../../linetools/spectra/tests/files/UM184_nF.fits')

In [13]:
# Set analysis range
abslin.limits.set([6080.78, 6087.82]*u.AA)  # wvlim

In [14]:
# Measure
abslin.measure_ew() # Observer frame
print('EW = {:g} with error {:g}'.format(abslin.attrib['EW'],abslin.attrib['sig_EW']))

EW = 0.993502 Angstrom with error 0.0527114 Angstrom


## Measure AODM

In [15]:
#
abslin.analy['spec'] = lsio.readspec('../../linetools/spectra/tests/files/UM184_nF.fits')
abslin.setz(2.92929)
abslin.limits.set((-150., 150.)*u.km/u.s)  # vlim

In [16]:
abslin.measure_aodm()
N, sigN, flgN = [abslin.attrib[key] for key in ['N','sig_N','flag_N']] 
print('logN = {:g}, siglogN = {:g}'.format(abslin.attrib['logN'], abslin.attrib['sig_logN']))

logN = 13.9051, siglogN = 0.0207026


### Another Table

In [17]:
tbl = ltlu.transtable_from_speclines(abslines)
tbl

wrest,name,Z,ion,Ej,z,EW,sig_EW,flag_N,logN,sig_logN
Angstrom,,,,1 / cm,,Angstrom,Angstrom,,,
float64,str8,int64,int64,float64,float64,float64,float64,int64,float64,float64
1215.67,HI 1215,1,1,0.0,0.0,0.0,0.0,0,0.0,0.0
1548.195,CIV 1548,6,4,0.0,2.92929,0.993502101206,0.0527114204241,1,13.9050897399,0.0207026007539
